In [1]:
import visa
from time import sleep, asctime, localtime, time
import pandas as pd
import datetime

#### Input parameters
data_points = 0
no_of_points_per_file = 30

time_delay = 120 # in seconds

is_source_current = False
source_current = 50e-9 # in A
voltage_limit = 200 # in V

is_source_voltage = True
source_voltage = 15 # in V
current_limit = 0.100 # in A

soak_time = 10 # in seconds

#### Save file location/name
csv_suffix = ''

date = datetime.date.today()
date_str = date.strftime("%d%B%Y")
csv_name = date_str + '_' + csv_suffix

#### Initialize list ####
current_list = []
voltage_list = []
time_list = []

#### Counters ####
counter = 0
file_count = 0
time_counter = 0
save_count = 1

### Connect to keithley
rm = visa.ResourceManager()
#print(rm.list_resources())
keithley = rm.open_resource('TCPIP0::169.254.0.1::inst0::INSTR')
print("Using " + keithley.query('*IDN?'))
sleep(1) # to ensure that the above prints properly

#Initialize SMU
keithley.write(" \
reset() \n\
errorqueue.clear() \n\
status.reset() \
")

if is_source_current is True:
    if is_source_voltage is True:
        print("Both is_source_current and is_source_voltage is set to True")
        print("Defaulting to source_current")
    #Configure source/measure functions
    keithley.write(" \
    smua.source.func = smua.OUTPUT_DCAMPS \n\
    smua.source.leveli = {levelI} \n\
    display.smua.measure.func = display.MEASURE_DCVOLTS \n\
    smua.measure.autozero = smua.AUTOZERO_ONCE \n\
    smua.measure.autorangev = smua.AUTORANGE_ON \n\
    smua.source.limitv = {limitV} \
    ".format(levelI = source_current, limitV = voltage_limit))

elif is_source_voltage is True:
    #Configure source/measure functions
    keithley.write(" \
    smua.source.func = smua.OUTPUT_DCVOLTS \n\
    smua.source.levelv = {levelV} \n\
    display.smua.measure.func = display.MEASURE_DCAMPS \n\
    smua.measure.autozero = smua.AUTOZERO_ONCE \n\
    smua.measure.autorangei = smua.AUTORANGE_ON \n\
    smua.source.limiti = {limitI} \
    ".format(levelV = source_voltage, limitI = current_limit))

print("Expt start time: " + asctime())
start_time = time()

#Turn on output
print("Turning on source {}".format("current" if is_source_current is True else "voltage"))
print("Source: {}".format(str(source_current) + " A" if is_source_current is True else str(source_voltage) + " V" ))
print("Limit: {}".format(str(voltage_limit) + " V" if is_source_current is True else str(current_limit) + " A" ))
print("")
keithley.write("smua.source.output = 1")

try:
    while 1:
        # Check if interlock is still enabled
        keithley.write("counter = errorqueue.count")
        error_count = keithley.query("print(counter)")[:-1]
        if float(error_count) > 0:
            keithley.write("errorId = errorqueue.next()")
            error_id = keithley.query("print(errorId)")[:-1]
            if float(error_id) == 8.02:
                raise AssertionError()
            print("Error occured: " + str(error_id))
            break

        # Increase time counter
        time_counter = time_counter + 1

        # Stop loop if max no of data points is reached
        if(counter > data_points):
            break

        # Increment only if data_point is defined i.e not 0
        if(data_points > 0):
            counter = counter + 1

        #Take measurements
        keithley.write("ireading, vreading = smua.measure.iv(smua.nvbuffer1, smua.nvbuffer2)")
        current = keithley.query("print(ireading)")[:-1] # to remove \n string at end
        voltage = keithley.query("print(vreading)")[:-1]

        current_list.append(current)
        voltage_list.append(voltage)
        time_list.append(asctime())
        # print out current time and measurements
        print(str(time_list[-1]) + ": " + str(voltage_list[-1]) + " | " + str(current_list[-1]))

        if(len(time_list) >= no_of_points_per_file * save_count):
            data = pd.DataFrame({
                'Voltage (V)': voltage_list,
                'Current (A)': current_list,
                'Time': time_list
            })

            data.to_csv(csv_name + str(file_count) + '.csv')
            print("\nFile save: " + csv_name + str(file_count) + '.csv\n')
            file_count = file_count + 1
            save_count = save_count + 1

        # Delay next measurement
        end_time = start_time + time_delay * time_counter
        while(end_time > time()):
            keithley.write("ireading, vreading = smua.measure.iv(smua.nvbuffer1, smua.nvbuffer2)")
            sleep(1)

except AssertionError:
    print("\n----------------------------")
    print("Output blocked by interlock!")
    print("----------------------------")
except:
    pass

# Save the data columns in a CSV file
data = pd.DataFrame({
    'Voltage (V)': voltage_list,
    'Current (A)': current_list,
    'Time': time_list
})

data.to_csv(csv_name + str(file_count) + '.csv')
print("\nFile save: " + csv_name + str(file_count) + '.csv')
print("\nExpt start time: " + asctime(localtime(start_time)))
print("Expt stop time: " + asctime())
print("Number of data points: " + str(time_counter-1))

#Turn off output and drain capacitors
if is_source_current is True:
    print("\nSetting source current to 0 A")
    keithley.write("smua.source.leveli = 0")
if is_source_voltage is True:
    print("\nSetting source voltage to 0 V")
    keithley.write("smua.source.levelv = 0")

print("Wait {}s until capacitors are fully discharged".format(soak_time))
sleep(soak_time)

print("Turning off output")
keithley.write("smua.source.output = 0")

print("Closing Keithley connection")
keithley.close()

Using Keithley Instruments Inc., Model 2657A, 4419521, 1.1.6

Expt start time: Fri Sep 13 14:18:30 2019
Turning on source voltage
Source: 15 V
Limit: 0.1 A

Fri Sep 13 14:18:30 2019: 1.49975e+01 | 3.71579e-05
Fri Sep 13 14:20:30 2019: 1.49999e+01 | 3.27860e-05
Fri Sep 13 14:22:31 2019: 1.50019e+01 | 3.24163e-05
Fri Sep 13 14:24:31 2019: 1.50029e+01 | 3.22056e-05
Fri Sep 13 14:26:31 2019: 1.50039e+01 | 3.20667e-05
Fri Sep 13 14:28:31 2019: 1.50046e+01 | 3.19674e-05
Fri Sep 13 14:30:31 2019: 1.50049e+01 | 3.18886e-05
Fri Sep 13 14:32:31 2019: 1.50056e+01 | 3.17842e-05
Fri Sep 13 14:34:31 2019: 1.50055e+01 | 3.16802e-05
Fri Sep 13 14:36:31 2019: 1.50058e+01 | 3.15905e-05
Fri Sep 13 14:38:31 2019: 1.50059e+01 | 3.15026e-05
Fri Sep 13 14:40:31 2019: 1.50063e+01 | 3.14207e-05
Fri Sep 13 14:42:31 2019: 1.50063e+01 | 3.13529e-05
Fri Sep 13 14:44:31 2019: 1.50064e+01 | 3.12859e-05
Fri Sep 13 14:46:31 2019: 1.50064e+01 | 3.12131e-05
Fri Sep 13 14:48:31 2019: 1.50069e+01 | 3.11411e-05
Fri Sep 13 

Fri Sep 13 19:22:30 2019: 1.50080e+01 | 2.74988e-05
Fri Sep 13 19:24:30 2019: 1.50085e+01 | 2.74633e-05
Fri Sep 13 19:26:30 2019: 1.50077e+01 | 2.74422e-05
Fri Sep 13 19:28:31 2019: 1.50081e+01 | 2.74270e-05
Fri Sep 13 19:30:31 2019: 1.50081e+01 | 2.74007e-05
Fri Sep 13 19:32:31 2019: 1.50082e+01 | 2.73794e-05
Fri Sep 13 19:34:31 2019: 1.50083e+01 | 2.73475e-05
Fri Sep 13 19:36:31 2019: 1.50079e+01 | 2.73265e-05
Fri Sep 13 19:38:31 2019: 1.50080e+01 | 2.73095e-05
Fri Sep 13 19:40:31 2019: 1.50082e+01 | 2.72888e-05
Fri Sep 13 19:42:31 2019: 1.50078e+01 | 2.72640e-05
Fri Sep 13 19:44:31 2019: 1.50079e+01 | 2.72437e-05
Fri Sep 13 19:46:31 2019: 1.50080e+01 | 2.72269e-05
Fri Sep 13 19:48:31 2019: 1.50081e+01 | 2.72123e-05
Fri Sep 13 19:50:31 2019: 1.50078e+01 | 2.71889e-05
Fri Sep 13 19:52:31 2019: 1.50079e+01 | 2.71689e-05
Fri Sep 13 19:54:31 2019: 1.50077e+01 | 2.71550e-05
Fri Sep 13 19:56:31 2019: 1.50083e+01 | 2.71265e-05
Fri Sep 13 19:58:31 2019: 1.50085e+01 | 2.71024e-05
Fri Sep 13 2

Sat Sep 14 00:32:31 2019: 1.50071e+01 | 2.53428e-05
Sat Sep 14 00:34:31 2019: 1.50073e+01 | 2.53549e-05
Sat Sep 14 00:36:30 2019: 1.50075e+01 | 2.53408e-05
Sat Sep 14 00:38:30 2019: 1.50075e+01 | 2.53515e-05
Sat Sep 14 00:40:30 2019: 1.50073e+01 | 2.53310e-05
Sat Sep 14 00:42:31 2019: 1.50073e+01 | 2.53414e-05
Sat Sep 14 00:44:31 2019: 1.50068e+01 | 2.53338e-05
Sat Sep 14 00:46:31 2019: 1.50069e+01 | 2.53263e-05
Sat Sep 14 00:48:31 2019: 1.50072e+01 | 2.53170e-05
Sat Sep 14 00:50:31 2019: 1.50075e+01 | 2.53084e-05
Sat Sep 14 00:52:31 2019: 1.50073e+01 | 2.53063e-05
Sat Sep 14 00:54:31 2019: 1.50072e+01 | 2.52913e-05
Sat Sep 14 00:56:31 2019: 1.50073e+01 | 2.52786e-05
Sat Sep 14 00:58:31 2019: 1.50070e+01 | 2.52932e-05
Sat Sep 14 01:00:31 2019: 1.50075e+01 | 2.52751e-05
Sat Sep 14 01:02:31 2019: 1.50074e+01 | 2.52873e-05
Sat Sep 14 01:04:31 2019: 1.50072e+01 | 2.52841e-05
Sat Sep 14 01:06:31 2019: 1.50073e+01 | 2.52780e-05
Sat Sep 14 01:08:31 2019: 1.50070e+01 | 2.52812e-05
Sat Sep 14 0

Sat Sep 14 05:42:31 2019: 1.50076e+01 | 2.50576e-05
Sat Sep 14 05:44:31 2019: 1.50079e+01 | 2.50449e-05
Sat Sep 14 05:46:31 2019: 1.50077e+01 | 2.50721e-05
Sat Sep 14 05:48:31 2019: 1.50079e+01 | 2.50785e-05
Sat Sep 14 05:50:31 2019: 1.50073e+01 | 2.50746e-05
Sat Sep 14 05:52:30 2019: 1.50077e+01 | 2.50697e-05
Sat Sep 14 05:54:30 2019: 1.50080e+01 | 2.50750e-05
Sat Sep 14 05:56:30 2019: 1.50079e+01 | 2.50760e-05
Sat Sep 14 05:58:31 2019: 1.50080e+01 | 2.50568e-05
Sat Sep 14 06:00:31 2019: 1.50074e+01 | 2.50429e-05
Sat Sep 14 06:02:31 2019: 1.50078e+01 | 2.50574e-05
Sat Sep 14 06:04:31 2019: 1.50074e+01 | 2.50676e-05
Sat Sep 14 06:06:31 2019: 1.50077e+01 | 2.50738e-05
Sat Sep 14 06:08:31 2019: 1.50079e+01 | 2.50649e-05
Sat Sep 14 06:10:31 2019: 1.50074e+01 | 2.50753e-05
Sat Sep 14 06:12:31 2019: 1.50082e+01 | 2.50872e-05
Sat Sep 14 06:14:31 2019: 1.50078e+01 | 2.50737e-05
Sat Sep 14 06:16:31 2019: 1.50077e+01 | 2.51066e-05

File save: 13September2019_15.csv

Sat Sep 14 06:18:31 2019: 1.

Sat Sep 14 10:52:31 2019: 1.50062e+01 | 2.51239e-05
Sat Sep 14 10:54:31 2019: 1.50066e+01 | 2.51154e-05
Sat Sep 14 10:56:31 2019: 1.50066e+01 | 2.51218e-05
Sat Sep 14 10:58:31 2019: 1.50066e+01 | 2.51310e-05
Sat Sep 14 11:00:31 2019: 1.50060e+01 | 2.51177e-05
Sat Sep 14 11:02:31 2019: 1.50065e+01 | 2.51124e-05
Sat Sep 14 11:04:31 2019: 1.50062e+01 | 2.51043e-05
Sat Sep 14 11:06:31 2019: 1.50059e+01 | 2.51113e-05
Sat Sep 14 11:08:31 2019: 1.50063e+01 | 2.51254e-05
Sat Sep 14 11:10:31 2019: 1.50064e+01 | 2.51238e-05
Sat Sep 14 11:12:30 2019: 1.50064e+01 | 2.51192e-05
Sat Sep 14 11:14:30 2019: 1.50063e+01 | 2.51187e-05
Sat Sep 14 11:16:30 2019: 1.50070e+01 | 2.51089e-05

File save: 13September2019_20.csv

Sat Sep 14 11:18:31 2019: 1.50065e+01 | 2.51147e-05
Sat Sep 14 11:20:31 2019: 1.50066e+01 | 2.50957e-05
Sat Sep 14 11:22:31 2019: 1.50066e+01 | 2.51067e-05
Sat Sep 14 11:24:31 2019: 1.50066e+01 | 2.51001e-05
Sat Sep 14 11:26:31 2019: 1.50066e+01 | 2.50932e-05
Sat Sep 14 11:28:31 2019: 1.

...

Tue Sep 17 16:18:30 2019: 1.50045e+01 | 2.35829e-05
Tue Sep 17 16:20:30 2019: 1.50046e+01 | 2.34777e-05
Tue Sep 17 16:22:31 2019: 1.50047e+01 | 2.34777e-05
Tue Sep 17 16:24:31 2019: 1.50045e+01 | 2.34471e-05
Tue Sep 17 16:26:31 2019: 1.50046e+01 | 2.34092e-05
Tue Sep 17 16:28:31 2019: 1.50049e+01 | 2.34160e-05
Tue Sep 17 16:30:31 2019: 1.50044e+01 | 2.34249e-05
Tue Sep 17 16:32:31 2019: 1.50045e+01 | 2.34065e-05
Tue Sep 17 16:34:31 2019: 1.50041e+01 | 2.33679e-05
Tue Sep 17 16:36:31 2019: 1.50046e+01 | 2.33739e-05
Tue Sep 17 16:38:31 2019: 1.50043e+01 | 2.33802e-05
Tue Sep 17 16:40:31 2019: 1.50044e+01 | 2.33696e-05
Tue Sep 17 16:42:31 2019: 1.50043e+01 | 2.33567e-05
Tue Sep 17 16:44:31 2019: 1.50046e+01 | 2.33498e-05
Tue Sep 17 16:46:31 2019: 1.50041e+01 | 2.33810e-05
Tue Sep 17 16:48:31 2019: 1.50043e+01 | 2.33739e-05
Tue Sep 17 16:50:31 2019: 1.50043e+01 | 2.33698e-05
Tue Sep 17 16:52:31 2019: 1.50043e+01 | 2.33858e-05
Tue Sep 17 16:54:31 2019: 1.50047e+01 | 2.33970e-05
Tue Sep 17 1

Tue Sep 17 21:28:30 2019: 1.50049e+01 | 2.37216e-05
Tue Sep 17 21:30:30 2019: 1.50053e+01 | 2.37385e-05
Tue Sep 17 21:32:30 2019: 1.50055e+01 | 2.37340e-05
Tue Sep 17 21:34:31 2019: 1.50048e+01 | 2.37287e-05
Tue Sep 17 21:36:31 2019: 1.50051e+01 | 2.37309e-05
Tue Sep 17 21:38:31 2019: 1.50052e+01 | 2.37199e-05
Tue Sep 17 21:40:31 2019: 1.50052e+01 | 2.37365e-05
Tue Sep 17 21:42:31 2019: 1.50054e+01 | 2.37394e-05
Tue Sep 17 21:44:31 2019: 1.50050e+01 | 2.37331e-05
Tue Sep 17 21:46:31 2019: 1.50050e+01 | 2.37331e-05
Tue Sep 17 21:48:31 2019: 1.50049e+01 | 2.37374e-05
Tue Sep 17 21:50:31 2019: 1.50054e+01 | 2.37294e-05
Tue Sep 17 21:52:31 2019: 1.50051e+01 | 2.37497e-05
Tue Sep 17 21:54:31 2019: 1.50054e+01 | 2.37585e-05
Tue Sep 17 21:56:31 2019: 1.50048e+01 | 2.37704e-05
Tue Sep 17 21:58:31 2019: 1.50051e+01 | 2.37534e-05
Tue Sep 17 22:00:31 2019: 1.50052e+01 | 2.37349e-05
Tue Sep 17 22:02:31 2019: 1.50051e+01 | 2.37357e-05
Tue Sep 17 22:04:31 2019: 1.50048e+01 | 2.37601e-05
Tue Sep 17 2

Wed Sep 18 02:36:31 2019: 1.50039e+01 | 2.36758e-05
Wed Sep 18 02:38:31 2019: 1.50041e+01 | 2.36803e-05
Wed Sep 18 02:40:30 2019: 1.50043e+01 | 2.37120e-05
Wed Sep 18 02:42:30 2019: 1.50041e+01 | 2.37113e-05
Wed Sep 18 02:44:30 2019: 1.50038e+01 | 2.37095e-05
Wed Sep 18 02:46:31 2019: 1.50044e+01 | 2.37384e-05
Wed Sep 18 02:48:31 2019: 1.50042e+01 | 2.37289e-05
Wed Sep 18 02:50:31 2019: 1.50037e+01 | 2.37245e-05
Wed Sep 18 02:52:31 2019: 1.50041e+01 | 2.37510e-05
Wed Sep 18 02:54:31 2019: 1.50037e+01 | 2.37689e-05
Wed Sep 18 02:56:31 2019: 1.50043e+01 | 2.37594e-05
Wed Sep 18 02:58:31 2019: 1.50040e+01 | 2.37563e-05
Wed Sep 18 03:00:31 2019: 1.50041e+01 | 2.38043e-05
Wed Sep 18 03:02:31 2019: 1.50042e+01 | 2.38550e-05
Wed Sep 18 03:04:31 2019: 1.50040e+01 | 2.38914e-05
Wed Sep 18 03:06:31 2019: 1.50041e+01 | 2.39013e-05
Wed Sep 18 03:08:31 2019: 1.50040e+01 | 2.38875e-05
Wed Sep 18 03:10:31 2019: 1.50043e+01 | 2.38664e-05
Wed Sep 18 03:12:31 2019: 1.50044e+01 | 2.38570e-05
Wed Sep 18 0

Wed Sep 18 07:44:31 2019: 1.50054e+01 | 2.35465e-05
Wed Sep 18 07:46:30 2019: 1.50055e+01 | 2.35469e-05
Wed Sep 18 07:48:30 2019: 1.50052e+01 | 2.35541e-05
Wed Sep 18 07:50:30 2019: 1.50053e+01 | 2.35596e-05
Wed Sep 18 07:52:31 2019: 1.50053e+01 | 2.35575e-05
Wed Sep 18 07:54:31 2019: 1.50051e+01 | 2.35671e-05
Wed Sep 18 07:56:31 2019: 1.50048e+01 | 2.35696e-05
Wed Sep 18 07:58:31 2019: 1.50051e+01 | 2.35601e-05
Wed Sep 18 08:00:31 2019: 1.50052e+01 | 2.35694e-05
Wed Sep 18 08:02:31 2019: 1.50053e+01 | 2.35660e-05
Wed Sep 18 08:04:31 2019: 1.50051e+01 | 2.35649e-05
Wed Sep 18 08:06:31 2019: 1.50054e+01 | 2.35644e-05
Wed Sep 18 08:08:31 2019: 1.50055e+01 | 2.35633e-05
Wed Sep 18 08:10:31 2019: 1.50054e+01 | 2.35645e-05
Wed Sep 18 08:12:31 2019: 1.50053e+01 | 2.35528e-05
Wed Sep 18 08:14:31 2019: 1.50050e+01 | 2.35522e-05
Wed Sep 18 08:16:31 2019: 1.50053e+01 | 2.35453e-05

File save: 13September2019_113.csv

Wed Sep 18 08:18:31 2019: 1.50052e+01 | 2.35524e-05
Wed Sep 18 08:20:31 2019: 1

Wed Sep 18 12:52:30 2019: 1.50039e+01 | 2.35607e-05
Wed Sep 18 12:54:31 2019: 1.50040e+01 | 2.35666e-05
Wed Sep 18 12:56:31 2019: 1.50038e+01 | 2.35703e-05
Wed Sep 18 12:58:31 2019: 1.50036e+01 | 2.35804e-05
Wed Sep 18 13:00:31 2019: 1.50044e+01 | 2.35732e-05
Wed Sep 18 13:02:31 2019: 1.50040e+01 | 2.35634e-05
Wed Sep 18 13:04:31 2019: 1.50041e+01 | 2.35937e-05
Wed Sep 18 13:06:31 2019: 1.50040e+01 | 2.35720e-05
Wed Sep 18 13:08:31 2019: 1.50035e+01 | 2.35707e-05
Wed Sep 18 13:10:31 2019: 1.50043e+01 | 2.35834e-05
Wed Sep 18 13:12:31 2019: 1.50042e+01 | 2.35881e-05
Wed Sep 18 13:14:31 2019: 1.50041e+01 | 2.35804e-05
Wed Sep 18 13:16:31 2019: 1.50037e+01 | 2.35757e-05

File save: 13September2019_118.csv

Wed Sep 18 13:18:31 2019: 1.50037e+01 | 2.35878e-05
Wed Sep 18 13:20:31 2019: 1.50038e+01 | 2.36017e-05
Wed Sep 18 13:22:31 2019: 1.50040e+01 | 2.36013e-05
Wed Sep 18 13:24:31 2019: 1.50039e+01 | 2.35743e-05
Wed Sep 18 13:26:31 2019: 1.50041e+01 | 2.35898e-05
Wed Sep 18 13:28:31 2019: 1

In [15]:
keithley.close()

In [6]:
### Connect to keithley
rm = visa.ResourceManager()
print(rm.list_resources())

('TCPIP0::169.254.0.1::inst0::INSTR', 'ASRL1::INSTR', 'ASRL2::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL9::INSTR', 'ASRL10::INSTR')


In [ ]:
keithley = rm.open_resource('TCPIP0::169.254.0.1::inst0::INSTR')

In [ ]:
import datetime

date = datetime.date.today()
date_str = date.strftime("%d%B%Y")
print(a)